<a href="https://colab.research.google.com/github/Rakitin11/handwrittenDigitsRec/blob/main/HandwrittenTextRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
from urllib.request import urlopen




In [23]:
# URLs
train_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra'
test_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes'

# Load data from URL
train_data = np.loadtxt(urlopen(train_url), delimiter=',')
test_data = np.loadtxt(urlopen(test_url), delimiter=',')

# Split features and labels
X_train = train_data[:, :-1]
y_train = train_data[:, -1].astype(int).reshape(-1, 1)

X_test = test_data[:, :-1]
y_test = test_data[:, -1].astype(int).reshape(-1, 1)

# Normalize pixel values (0-16)
X_train = X_train / 16.0
X_test = X_test / 16.0


In [24]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
y_train_oh = encoder.fit_transform(y_train)
y_test_oh = encoder.transform(y_test)


In [25]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-8), axis=1))

def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))


In [26]:
np.random.seed(42)

input_size = X_train.shape[1]  # should be 64
hidden_size = 64
output_size = 10
learning_rate = 0.01
epochs = 1000

W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
b2 = np.zeros((1, output_size))


In [27]:
for epoch in range(epochs):
    # Forward pass
    z1 = np.dot(X_train, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = softmax(z2)

    # Loss and accuracy
    loss = cross_entropy(y_train_oh, a2)
    acc = accuracy(y_train_oh, a2)

    # Backpropagation
    delta2 = (a2 - y_train_oh) / X_train.shape[0]
    dW2 = np.dot(a1.T, delta2)
    db2 = np.sum(delta2, axis=0, keepdims=True)

    delta1 = np.dot(delta2, W2.T) * relu_derivative(a1)
    dW1 = np.dot(X_train.T, delta1)
    db1 = np.sum(delta1, axis=0, keepdims=True)

    # Update weights
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

    # Print every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{epochs} - Loss: {loss:.4f} - Accuracy: {acc:.4f}')



Epoch 10/1000 - Loss: 2.3120 - Accuracy: 0.1310
Epoch 20/1000 - Loss: 2.2634 - Accuracy: 0.1525
Epoch 30/1000 - Loss: 2.2189 - Accuracy: 0.1807
Epoch 40/1000 - Loss: 2.1775 - Accuracy: 0.2161
Epoch 50/1000 - Loss: 2.1381 - Accuracy: 0.2595
Epoch 60/1000 - Loss: 2.1004 - Accuracy: 0.2943
Epoch 70/1000 - Loss: 2.0639 - Accuracy: 0.3291
Epoch 80/1000 - Loss: 2.0285 - Accuracy: 0.3597
Epoch 90/1000 - Loss: 1.9940 - Accuracy: 0.3971
Epoch 100/1000 - Loss: 1.9602 - Accuracy: 0.4240
Epoch 110/1000 - Loss: 1.9269 - Accuracy: 0.4549
Epoch 120/1000 - Loss: 1.8942 - Accuracy: 0.4844
Epoch 130/1000 - Loss: 1.8620 - Accuracy: 0.5109
Epoch 140/1000 - Loss: 1.8303 - Accuracy: 0.5394
Epoch 150/1000 - Loss: 1.7989 - Accuracy: 0.5598
Epoch 160/1000 - Loss: 1.7681 - Accuracy: 0.5828
Epoch 170/1000 - Loss: 1.7377 - Accuracy: 0.6006
Epoch 180/1000 - Loss: 1.7077 - Accuracy: 0.6181
Epoch 190/1000 - Loss: 1.6781 - Accuracy: 0.6375
Epoch 200/1000 - Loss: 1.6488 - Accuracy: 0.6552
Epoch 210/1000 - Loss: 1.6200

In [28]:
# Forward pass on test data
z1_test = np.dot(X_test, W1) + b1
a1_test = relu(z1_test)
z2_test = np.dot(a1_test, W2) + b2
a2_test = softmax(z2_test)

# Evaluate
test_loss = cross_entropy(y_test_oh, a2_test)
test_acc = accuracy(y_test_oh, a2_test)

print(f'\nTest Loss: {test_loss:.4f} - Test Accuracy: {test_acc:.4f}')



Test Loss: 0.5821 - Test Accuracy: 0.8843
